In [1]:
import os,cv2,random,numpy as np

In [3]:
def load_images_from_directory(directory, target_size=(256, 128)):
    """
    Load images from a directory and return a list of dictionaries containing filename, image, and filepath.
    """
    data = []

    for root, dirs, _ in os.walk(directory):
        for _, _, files in os.walk(root):
            for filename in files:
                if filename.endswith(".jpg") or filename.endswith(".png"):
                    filepath = os.path.join(root, filename)
                    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        img_resized = cv2.resize(img, target_size)
                        user_id = filename.split('_')[0]  # Extract user ID from the filename

                        data.append({'filename': filename, 'user_id': user_id, 'image': img_resized, 'filepath': filepath})

    return data


In [4]:
def apply_rotations_and_save(images_data, output_dir="output", num_per_user=5, rotation_angles=[45, 90, 135]):
    user_data = {}

    # Group images by user
    for img_data in images_data:
        user_id = img_data['user_id']
        if user_id not in user_data:
            user_data[user_id] = []
        user_data[user_id].append(img_data)

    # Apply rotations and save
    for user_id, images in user_data.items():
        selected_images = random.sample(images, min(len(images), num_per_user))

        for img_data in selected_images:
            img = img_data['image']
            filename = img_data['filename']

            # Randomly select an angle from the provided list of angles
            angle = random.choice(rotation_angles)

            # Get the image center and dimensions
            h, w = img.shape
            center = (w // 2, h // 2)

            # Calculate the rotation matrix
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

            # Compute the new bounding box dimensions to prevent cropping
            abs_cos = abs(rotation_matrix[0, 0])
            abs_sin = abs(rotation_matrix[0, 1])
            new_w = int(2*h * abs_sin + w * abs_cos)
            new_h = int(2*h * abs_cos + w * abs_sin)

            # Adjust the rotation matrix to account for the translation
            rotation_matrix[0, 2] += (new_w / 2) - center[0]
            rotation_matrix[1, 2] += (new_h / 2) - center[1]

            # Perform the rotation with the new bounding box size
            rotated_img = cv2.warpAffine(img, rotation_matrix, (new_w, new_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)

            # Resize the rotated image back to the original dimensions
            resized_img = cv2.resize(rotated_img, (w, h))

            # Save the image in an angle-based folder
            angle_dir = os.path.join(output_dir, f"angle_{angle}")
            os.makedirs(angle_dir, exist_ok=True)

            output_path = os.path.join(angle_dir, filename)
            cv2.imwrite(output_path, resized_img)

    print(f"Images saved to {output_dir}")

In [8]:
# Define paths
directory = "data/isolated_words_per_user"
output_dir = "preprocessing/rotated_images"

# Load images
loaded_data = load_images_from_directory(directory, target_size=(256, 128))

rotation_angles = [45, 90, 135]
apply_rotations_and_save(loaded_data, output_dir=output_dir, num_per_user=5, rotation_angles=rotation_angles)



Images saved to preprocessing/rotated_images


In [5]:
def add_gaussian_noise(image, mean=0, std=10):
    noise = np.random.normal(mean, std, image.shape).astype(np.float32)
    noisy_image = image.astype(np.float32) + noise

    # Clip the pixel values to be in the valid range [0, 255]
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return noisy_image

In [6]:
def apply_noise_and_save(images_data, output_dir="output_noise", num_per_user=5, noise_levels=[10, 20, 30]):
    user_data = {}

    # Group images by user
    for img_data in images_data:
        user_id = img_data['user_id']
        if user_id not in user_data:
            user_data[user_id] = []
        user_data[user_id].append(img_data)

    # Apply noise and save
    for user_id, images in user_data.items():
        # Select `num_per_user` images for each user
        selected_images = random.sample(images, min(len(images), num_per_user))

        for img_data in selected_images:
            img = img_data['image']
            filename = img_data['filename']

            # Randomly select a noise level from the provided list
            noise_std = random.choice(noise_levels)

            # Add Gaussian noise to the image
            noisy_img = add_gaussian_noise(img, mean=0, std=noise_std)

            # Save the noisy image in a noise-level-based folder
            noise_dir = os.path.join(output_dir, f"noise_{noise_std}")
            os.makedirs(noise_dir, exist_ok=True)

            output_path = os.path.join(noise_dir, filename)
            cv2.imwrite(output_path, noisy_img)

    print(f"Noisy images saved to {output_dir}")

In [7]:
images_data = load_images_from_directory("data/isolated_words_per_user", target_size=(256, 128))

# Apply noise and save
apply_noise_and_save(images_data, output_dir="preprocessing/noisy_images", num_per_user=5, noise_levels=[10, 20, 30])

Noisy images saved to preprocessing/noisy_images
